# Gridded data review

**Description:**   
This script is part of the annual gridded data review for CLRTAP 

**Original Author:** Christopher Evangelides   
**Date:**  May 2024   
**Maintainer:** XXX      
**Date:** XXX 20XX    

In [10]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as db
from sqlalchemy import create_engine
import os
import geopandas as gpd

In [11]:
user = os.getlogin()

ROOT_PSW = os.path.join(r'C:\Users',user,"db_credentials")
filename_sqlalchemy = 'sqlalchemy_engine_gisdata.txt'
filename_sqlalchemy_path = os.path.join(ROOT_PSW, filename_sqlalchemy)

# Load the credentials with the correct path
if os.path.exists(filename_sqlalchemy_path):
    print("Your are: " + user)

    with open(filename_sqlalchemy_path) as f:
        lines = f.readlines()
        engine = db.create_engine(lines[0])

Your are: ce6
